📢 [프롬프트 엔지니어링, LLM(ChatGPT) 기반의 AI 서비스 개발](https://fastcampus.co.kr/data_red_golbin) 강의에서 사용하는 예제입니다.

### 데이터 생성 프롬프트

```
첨부한 기고문에서 질문과 답변 세트를 만들어줘

--- 기고문 ---
{article}
--- 기고문 끝 ---

질문과 답변 세트를 5개 만들어줘.
각 질문과 답변은 전부 다른 내용을 담고 있어야 해.
숫자로 반복문을 돌리지 말고, 첨부한 파일의 내용에 있는 내용을 구체적으로 담아야 해.

질문 답변 필드명은 instruction과 output으로 작성해줘
질문과 답변은 한국어로 작성해줘

출력 형식은 CSV로 출력해줘. 각 항목은 쌍따옴표로 감싸줘.
```

생성한 파일

- CSV 데이터 파일: [the-age-of-AI-has-begun.csv](https://drive.google.com/file/d/13G40JFsrSJB8GgZI4AiC2s0yF3tmVQCj/view?usp=sharing)
- 기고문 텍스트 파일: [The Age of AI has begun](https://docs.google.com/document/d/1nXYbAJRYzQ9To1pXscFZ10YU_5G2A1N-ml4_kbaGIJY/edit?usp=sharing)

In [1]:
!nvidia-smi

Wed Oct  4 08:24:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 모델 로딩

In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

## 데이터셋 로딩

In [21]:
from datasets import load_dataset

data = load_dataset("csv",
                    data_files={"train":"한국법제연구원_법령용어 정보_20200831.csv"})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
[row for row in data['train']]

[{'연번': 1,
  '한글용어': '흡수합병',
  '한자용어': '吸收合倂',
  '용어설명': '법률이 정하는 절차에 의하여 2 이상의 법인 전부 또는 그중 1개의 법인이외의 법인이 해산하여 청산절차를 행하지 않고 소멸하고, 소멸된 법인의 권리의무가 새로이 설립되는 법인 또는 존속하는 법인에게 포괄적으로 승계되는 것을 합병이라고 하며, 합병의 당사자인 법인 중 1법인이 존속하는 경우의 합병을 말한다.'},
 {'연번': 2,
  '한글용어': '휴회',
  '한자용어': '休會',
  '용어설명': '의회가 회기 중 일시 활동을 그치는 것을 말한다. 휴회는 폐회와 달리 휴회기간이 경과하면 의회의 활동은 휴회 전에 이어 다시 개시된다.'},
 {'연번': 3,
  '한글용어': '휴직',
  '한자용어': '休職',
  '용어설명': '휴직제도는 일반적으로 일정한 기간동안 해당직무를 쉬는 것을 말하는 바, 공무원의 경우에 있어서는 신분은 보유하나 직무에는 종사하지 못하는 임용행위의 일종을 말한다. 휴직은 공무원에 대한 불이익처분이므로 법률에 정하는 사유가 있는 경우에 한하여 직권휴직이 인정되며, 그 밖에 일정한 사유가 있는 경우에 본인이 원하여 인정하는 의원휴직이 있다. 또한, 남녀고용평등법에 의하여 생후 1년 미만의 영아를 가진 근로자가 그 영아의 양육을 위하여 육아휴직을 인정하고 있다.'},
 {'연번': 4,
  '한글용어': '휴업수당',
  '한자용어': '休業手當',
  '용어설명': '근로기준법상 사용자의 귀책사유로 인하여 휴업하는 경우에는 사용자는 휴업기간 중 당해 근로자에 대하여 지급하는 수당을 말한다. 사용자는 휴업기간 중 근로자에 대하여 평균임금의 100분의 70 이상의 휴업수당을 지급하여야 한다. 다만, 평균임금의 100분의 70에 상당하는 금액이 통상임금을 초과하는 경우에는 통상임금을 휴업수당으로 지급할 수 있다.'},
 {'연번': 5,
  '한글용어': '휴업보상',
  '한자용어': '休業補償',
  '용어설명': '재해보상의

In [23]:
data = data.map(
    lambda x: {
        'text': f"""User: {x['한글용어']}

Assistant: {x['용어설명']}<|endoftext|>""" }
)

Map:   0%|          | 0/6178 [00:00<?, ? examples/s]

In [24]:
[row['text'] for row in data['train']]

['User: 흡수합병\n\nAssistant: 법률이 정하는 절차에 의하여 2 이상의 법인 전부 또는 그중 1개의 법인이외의 법인이 해산하여 청산절차를 행하지 않고 소멸하고, 소멸된 법인의 권리의무가 새로이 설립되는 법인 또는 존속하는 법인에게 포괄적으로 승계되는 것을 합병이라고 하며, 합병의 당사자인 법인 중 1법인이 존속하는 경우의 합병을 말한다.<|endoftext|>',
 'User: 휴회\n\nAssistant: 의회가 회기 중 일시 활동을 그치는 것을 말한다. 휴회는 폐회와 달리 휴회기간이 경과하면 의회의 활동은 휴회 전에 이어 다시 개시된다.<|endoftext|>',
 'User: 휴직\n\nAssistant: 휴직제도는 일반적으로 일정한 기간동안 해당직무를 쉬는 것을 말하는 바, 공무원의 경우에 있어서는 신분은 보유하나 직무에는 종사하지 못하는 임용행위의 일종을 말한다. 휴직은 공무원에 대한 불이익처분이므로 법률에 정하는 사유가 있는 경우에 한하여 직권휴직이 인정되며, 그 밖에 일정한 사유가 있는 경우에 본인이 원하여 인정하는 의원휴직이 있다. 또한, 남녀고용평등법에 의하여 생후 1년 미만의 영아를 가진 근로자가 그 영아의 양육을 위하여 육아휴직을 인정하고 있다.<|endoftext|>',
 'User: 휴업수당\n\nAssistant: 근로기준법상 사용자의 귀책사유로 인하여 휴업하는 경우에는 사용자는 휴업기간 중 당해 근로자에 대하여 지급하는 수당을 말한다. 사용자는 휴업기간 중 근로자에 대하여 평균임금의 100분의 70 이상의 휴업수당을 지급하여야 한다. 다만, 평균임금의 100분의 70에 상당하는 금액이 통상임금을 초과하는 경우에는 통상임금을 휴업수당으로 지급할 수 있다.<|endoftext|>',
 'User: 휴업보상\n\nAssistant: 재해보상의 일종으로서, 요양으로 인하여 근로자가 노동불능이 되어 임금을 받을 수 없게 된 때에 일정한 경우를 제외하고는 사용자가 근로자의 요양 중 그 근로자의 평균임금의 100분의 60에 해당하

## 텍스트 데이터 tokenize

In [25]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/6178 [00:00<?, ? examples/s]

In [26]:
data["train"][0]["text"]

'User: 흡수합병\n\nAssistant: 법률이 정하는 절차에 의하여 2 이상의 법인 전부 또는 그중 1개의 법인이외의 법인이 해산하여 청산절차를 행하지 않고 소멸하고, 소멸된 법인의 권리의무가 새로이 설립되는 법인 또는 존속하는 법인에게 포괄적으로 승계되는 것을 합병이라고 하며, 합병의 당사자인 법인 중 1법인이 존속하는 경우의 합병을 말한다.<|endoftext|>'

In [27]:
tokenized_text = tokenizer(data["train"][0]["text"])
tokenized_text

{'input_ids': [56, 426, 85, 29, 6709, 9444, 202, 202, 36, 12732, 15097, 15741, 29, 3558, 270, 5620, 272, 2822, 274, 9709, 520, 401, 958, 285, 3948, 4558, 2259, 353, 653, 376, 533, 285, 3948, 270, 872, 285, 3948, 270, 11545, 13473, 8198, 5892, 301, 24639, 295, 536, 283, 11708, 4296, 15, 11708, 551, 3948, 285, 4456, 15617, 293, 26268, 2344, 605, 272, 3948, 2259, 21721, 284, 272, 3948, 666, 10107, 10562, 12512, 605, 272, 388, 276, 5848, 1420, 423, 453, 15, 5848, 285, 12404, 2205, 3948, 469, 376, 3429, 270, 21721, 284, 272, 955, 285, 5848, 276, 502, 660, 17, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [28]:
token_text = [tokenizer.decode(id) for id in tokenized_text["input_ids"]]
", ".join(token_text)

'U, se, r, :,  흡수, 합병, \n, \n, A, ss, ist, ant, :,  법률, 이,  정하, 는,  절차, 에,  의하, 여,  2,  이상, 의,  법인,  전부,  또는,  그, 중,  1, 개, 의,  법인, 이, 외, 의,  법인, 이,  해산, 하여,  청산, 절차, 를,  행하, 지,  않, 고,  소멸, 하고, ,,  소멸, 된,  법인, 의,  권리, 의무, 가,  새로이,  설립, 되, 는,  법인,  또는,  존속, 하, 는,  법인, 에게,  포괄, 적으로,  승계, 되, 는,  것, 을,  합병, 이라고,  하, 며, ,,  합병, 의,  당사, 자인,  법인,  중,  1, 법인, 이,  존속, 하, 는,  경우, 의,  합병, 을,  말, 한다, ., <|endoftext|>'

## PEFT 학습 준비

In [29]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [30]:
# 학습되는 파라미터 수 출력
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"""trainable params: {trainable_params}
all params: {all_param}
trainable%: {100 * trainable_params / all_param}""")

trainable params: 6553600
all params: 6608701440
trainable%: 0.09916622894073424


## 학습하기

In [31]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        max_steps=150, # 100 step 학습시, V100 기준 약 2분 소요
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        learning_rate=1e-4,
        fp16=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        logging_steps=25,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.135100
50,1.980400
75,1.755600
100,1.557400
125,1.602400
150,1.728600


TrainOutput(global_step=150, training_loss=1.7932364145914714, metrics={'train_runtime': 189.166, 'train_samples_per_second': 1.586, 'train_steps_per_second': 0.793, 'total_flos': 4164625360650240.0, 'train_loss': 1.7932364145914714, 'epoch': 0.05})

In [32]:
model.eval()
model.config.use_cache = True

In [33]:
from transformers.generation import GenerationConfig

gen_cfg = GenerationConfig.from_model_config(model.config)
gen_cfg.pad_token_id = tokenizer.eos_token_id

def gen(x):
    gen_cfg.temperature = 0.1
    gen_cfg.max_new_tokens = 128

    gened = model.generate(
        **tokenizer(
            f"User: {x}\n\nAssistant:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        do_sample=True,
        generation_config=gen_cfg
    )

    return tokenizer.decode(gened[0])

In [35]:
gen('흡수합병이란?')

'User: 흡수합병이란?\n\nAssistant: 흡수합병이란 2개 이상의 회사가 하나의 회사로 되는 것을 말한다. 흡수합병의 경우에는 합병 후 존속하는 회사 또는 합병에 의하여 설립되는 회사는 합병에 의하여 소멸하는 회사의 권리와 의무를 승계한다. 흡수합병의 경우에는 합병에 의하여 소멸하는 회사의 주주에게 합병대가를 지급하지 않는다. 흡수합병의 경우에는 합병에 의하여 소멸하는 회사의 주주에게 합병대가를 지급하지 않는다. 흡수합병의 경우에는 합병에 의하여 소멸하는 회사의 주주에게 합병'

endoftext 이 안나왔는데 나오려면 1000개 이상의 데이터셋을 학습시켜야 함

## 모델 저장 & 업로드

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# !huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
# model.push_to_hub('계정명/koalpaca-polyglot-12.8b-bill')

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/golbin/koalpaca-polyglot-12.8b-bill/commit/b5591d3d4fda960320dfa2fa26da2676120d2207', commit_message='Upload model', commit_description='', oid='b5591d3d4fda960320dfa2fa26da2676120d2207', pr_url=None, pr_revision=None, pr_num=None)

Reference: [KoAlpaca QLoRA Training Original Notebook by Beomi](https://colab.research.google.com/gist/Beomi/a3032e4eaa33b86fdf8de1f47f15a647/2023_05_26_bnb_4bit_koalpaca_v1_1a_on_polyglot_ko_12_8b.ipynb)